# DropBlock: Nedir? Nasıl Tanımlanır? Modele Nasıl Entegre Edilir? (PyTorch)

Bu notebook; **DropBlock**’un kısa tanımını, arkasındaki fikri ve **en temelden ileri seviyeye** PyTorch ile nasıl yazılıp **modele adım adım entegre edileceğini** gösterir.

> Not: Kodlar `torch>=1.10` ile uyumludur. GPU varsa otomatik kullanır.


## 1) DropBlock kısaca nedir?

**Dropout** nöronları (tekil aktivasyonları) rastgele sıfırlarken, **DropBlock** özellikle CNN’lerde **özellik haritalarında (feature map)** **bitişik bir blok alanı** rastgele sıfırlar.  
Amaç: Modelin **lokal ipuçlarına aşırı bağımlı** olmasını azaltmak, **daha sağlam (robust) ve genelleştirici** temsiller öğrenmesini sağlamak.

**Neden CNN’de Dropout zayıf kalabiliyor?**  
CNN feature map’lerinde komşu pikseller/aktivasyonlar güçlü korelasyon taşır. Tek tek aktivasyon düşürmek (Dropout) çoğu zaman “yakın komşular” tarafından telafi edilir. DropBlock ise **komşu bir bölgeyi birlikte düşürerek** bu telafiyi zorlaştırır.

**Kilit fikir:**  
- Dropout: noktasal maskeleme  
- DropBlock: **blok (patch) maskeleme** ✅


## 2) Temel kavramlar: `drop_prob`, `block_size`, `gamma`

DropBlock’ta iki ana hiperparametre var:

- `drop_prob`: “Ne kadar” düşüreyim? (0.0–0.5 arası sık kullanılır)
- `block_size`: “Ne büyüklükte blok” düşüreyim? (ör: 3, 5, 7)

Uygulamada genelde şu yapılır:

1. Önce **blok merkezlerini** Bernoulli ile örnekle (olasılık `gamma`).
2. Bu merkezlerden `block_size x block_size` alanı **1→0** olacak şekilde genişlet (max-pool/dilation benzeri).
3. Maskeyi uygula ve aktivasyonları **yeniden ölçekle**:  
   Beklenen aktivasyon büyüklüğü korunur.

> `gamma`, `drop_prob`’dan türetilir. Çünkü merkez sayısı ile blok alanı farklı şeylerdir.


## 3) En temel DropBlock: 2D feature map için (PyTorch)

Aşağıda “klasik” yaklaşımı yazacağız:
- Sadece **eğitim modunda** çalışır (`self.training`).
- `gamma` hesaplar.
- Maske üretir.
- Bloklara genişletir.
- `x * mask * scale` uygular.

> Not: Bu implementasyon, pratikte yaygın kullanılan ve anlaşılır bir sürümdür.


In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
class DropBlock2D(nn.Module):
    def __init__(self, drop_prob: float = 0.1, block_size: int = 7):
        super().__init__()
        self.drop_prob = float(drop_prob)
        self.block_size = int(block_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Eğitim dışındaysa veya drop_prob 0 ise hiçbir şey yapma
        if (not self.training) or self.drop_prob <= 0.0:
            return x

        # x: (N, C, H, W)
        if x.dim() != 4:
            raise ValueError(f"DropBlock2D 4D tensor bekler (N,C,H,W). Geldi: {x.shape}")

        n, c, h, w = x.shape
        bs = self.block_size

        # Block_size, H/W'den büyük olursa pratikte tüm alanı düşürmeye kayar.
        # Bu yüzden güvenli tarafta clamp yapıyoruz.
        bs = min(bs, h, w)
        if bs < 1:
            return x

        # Gamma: blok merkezlerinin seçilme olasılığı.
        # Yaklaşık fikir: hedeflenen toplam düşen alan oranını drop_prob yap.
        # (H*W) yerine "seçilebilir merkez" alanı (H-bs+1, W-bs+1) üzerinden düzeltme.
        valid_h = h - bs + 1
        valid_w = w - bs + 1

        # block alanı
        block_area = bs * bs

        # gamma (literatürde yaygın form)
        gamma = self.drop_prob * (h * w) / (block_area * valid_h * valid_w)

        # 1) Merkez maskesi (N,C,valid_h,valid_w) üzerinde örnekle
        # Sonra bunu (N,C,H,W)'ye pad ile yerleştiriyoruz.
        center_mask = (torch.rand((n, c, valid_h, valid_w), device=x.device) < gamma).float()

        # 2) Merkez maskesini HxW boyuta taşı (pad)
        pad_h = (bs - 1) // 2
        pad_w = (bs - 1) // 2
        center_mask = F.pad(center_mask, (pad_w, pad_w, pad_h, pad_h))

        # 3) Bloklara genişlet: max_pool ile "yakın çevrede merkez varsa 1" olur
        # kernel=bs, stride=1, padding=bs//2 => HxW korunur
        block_mask = F.max_pool2d(center_mask, kernel_size=bs, stride=1, padding=bs//2)

        # 4) Drop mask = 1 - block_mask (düşürülen yerler 0 olur)
        mask = 1.0 - block_mask

        # 5) Yeniden ölçekleme: beklenen aktivasyon büyüklüğünü koru
        # mask ortalaması ~ keep_prob => scale = 1/keep_prob
        keep_prob = mask.mean().clamp(min=1e-6)
        x = x * mask / keep_prob
        return x

### Hızlı test

- Eğitim modunda (`train()`) çıktıda bloklu sıfırlamalar görmelisin.
- `eval()` modunda aynı tensor geri dönmeli.


In [4]:
torch.manual_seed(0)

m = DropBlock2D(drop_prob=0.25, block_size=5).to(device)
x = torch.randn(2, 3, 16, 16, device=device)

m.train()
y_train = m(x)

m.eval()
y_eval = m(x)

# Farklara bak
print("train fark (L1):", (y_train - x).abs().mean().item())
print("eval fark  (L1):", (y_eval - x).abs().mean().item())


train fark (L1): 0.33283552527427673
eval fark  (L1): 0.0


## 4) DropBlock’u “en temelden ileriye” doğru güçlendirme

Aşağıdaki iyileştirmeler pratikte çok işe yarar:

1. **Block size otomatik ayarlanabilir**: erken katmanlarda küçük, derin katmanlarda büyük blok.
2. **Linear schedule** (drop_prob ramp-up): eğitim başında düşük, sonlara doğru artan drop_prob.
3. **Feature map boyutuna duyarlı**: küçük HxW’de agresif drop_prob patlatabilir.
4. **Determinism / seed kontrolü**: debug sırasında.

Şimdi bunları paketleyelim.


In [5]:
class DropBlock2D_Scheduled(nn.Module):
    """DropBlock2D + lineer schedule.

    Notlar:
    - forward'da step alabilir veya dışarıdan set edebilirsin.
    - drop_prob, [0, max_drop_prob] aralığında lineer artar.

    Args:
        max_drop_prob: Eğitim sonunda ulaşılacak drop_prob.
        block_size: Sabit blok boyutu.
        total_steps: Schedule toplam adım sayısı.
    """
    def __init__(self, max_drop_prob=0.1, block_size=7, total_steps=10_000):
        super().__init__()
        self.max_drop_prob = float(max_drop_prob)
        self.block_size = int(block_size)
        self.total_steps = int(total_steps)
        self.register_buffer("step", torch.zeros((), dtype=torch.long))

    def _current_drop_prob(self):
        s = int(self.step.item())
        t = min(max(s / max(1, self.total_steps), 0.0), 1.0)
        return self.max_drop_prob * t

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.training:
            self.step += 1

        drop_prob = self._current_drop_prob()
        # Aynı DropBlock mantığı, sadece drop_prob dinamik
        if (not self.training) or drop_prob <= 0.0:
            return x

        n, c, h, w = x.shape
        bs = min(self.block_size, h, w)
        valid_h = h - bs + 1
        valid_w = w - bs + 1
        block_area = bs * bs

        gamma = drop_prob * (h * w) / (block_area * valid_h * valid_w)

        center_mask = (torch.rand((n, c, valid_h, valid_w), device=x.device) < gamma).float()
        pad_h = (bs - 1) // 2
        pad_w = (bs - 1) // 2
        center_mask = F.pad(center_mask, (pad_w, pad_w, pad_h, pad_h))
        block_mask = F.max_pool2d(center_mask, kernel_size=bs, stride=1, padding=bs//2)
        mask = 1.0 - block_mask
        keep_prob = mask.mean().clamp(min=1e-6)
        return x * mask / keep_prob


## 5) DropBlock’u modele entegre etme: adım adım

Aşağıdaki akış “temiz” entegrasyon yaklaşımıdır:

1. **Conv → Norm → Act** bloğunu yaz
2. İstediğin yerlerde **DropBlock** ekle
3. Modeli oluştur
4. Eğitim döngüsünde `model.train()` iken DropBlock aktif olur, `eval()` iken pasif olur

Şimdi küçük ama “ciddi” bir CNN yazalım: Residual + DropBlock.


In [6]:
class ConvGNAct(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1, groups_gn=8, act="silu"):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False)
        # GroupNorm: batch size küçükken daha stabil
        g = min(groups_gn, cout)
        # g, cout'u bölmeli; değilse en yakın böleni seç
        while cout % g != 0 and g > 1:
            g -= 1
        self.norm = nn.GroupNorm(g, cout)
        if act == "silu":
            self.act = nn.SiLU(inplace=True)
        elif act == "relu":
            self.act = nn.ReLU(inplace=True)
        else:
            raise ValueError("act 'silu' veya 'relu' olmalı")

    def forward(self, x):
        return self.act(self.norm(self.conv(x)))


class ResidualBlock(nn.Module):
    def __init__(self, cin, cout, stride=1, dropblock: nn.Module | None = None):
        super().__init__()
        self.dropblock = dropblock

        self.conv1 = ConvGNAct(cin, cout, k=3, s=stride, p=1)
        self.conv2 = nn.Conv2d(cout, cout, 3, padding=1, bias=False)
        g = min(8, cout)
        while cout % g != 0 and g > 1:
            g -= 1
        self.norm2 = nn.GroupNorm(g, cout)

        self.skip = None
        if stride != 1 or cin != cout:
            self.skip = nn.Conv2d(cin, cout, 1, stride=stride, bias=False)

        self.act = nn.SiLU(inplace=True)

    def forward(self, x):
        identity = x if self.skip is None else self.skip(x)

        out = self.conv1(x)

        # DropBlock'u genelde conv sonrası feature map üzerinde uygularsın
        if self.dropblock is not None:
            out = self.dropblock(out)

        out = self.norm2(self.conv2(out))
        out = out + identity
        return self.act(out)


In [7]:
class SmallResNetDropBlock(nn.Module):
    def __init__(self, num_classes=10, dropblock_cfg=None):
        super().__init__()

        # DropBlock'u stage bazlı koymak mantıklı:
        # - Çok erken katmanda agresif dropblock bazen performansı düşürür.
        # - Orta/derin katmanlar daha iyi aday.
        dropblock_cfg = dropblock_cfg or {}
        db1 = dropblock_cfg.get("stage1", None)
        db2 = dropblock_cfg.get("stage2", None)
        db3 = dropblock_cfg.get("stage3", None)

        self.stem = ConvGNAct(3, 64, k=3, s=1, p=1)

        self.stage1 = nn.Sequential(
            ResidualBlock(64, 64, stride=1, dropblock=db1),
            ResidualBlock(64, 64, stride=1, dropblock=db1),
        )
        self.stage2 = nn.Sequential(
            ResidualBlock(64, 128, stride=2, dropblock=db2),
            ResidualBlock(128, 128, stride=1, dropblock=db2),
        )
        self.stage3 = nn.Sequential(
            ResidualBlock(128, 256, stride=2, dropblock=db3),
            ResidualBlock(256, 256, stride=1, dropblock=db3),
        )

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.pool(x).flatten(1)
        return self.fc(x)


### 5.1) Modeli DropBlock olmadan / DropBlock ile kur

Aşağıda iki model kuruyoruz:
- Baseline (DropBlock yok)
- DropBlock’lu (stage2+stage3)

> İpucu: DropBlock’u çoğunlukla **orta ve derin stage**’lerde başlatmak daha güvenli.


In [8]:
baseline = SmallResNetDropBlock(num_classes=10, dropblock_cfg={}).to(device)

dropblock_cfg = {
    "stage2": DropBlock2D_Scheduled(max_drop_prob=0.15, block_size=5, total_steps=2000),
    "stage3": DropBlock2D_Scheduled(max_drop_prob=0.20, block_size=7, total_steps=2000),
}
model_db = SmallResNetDropBlock(num_classes=10, dropblock_cfg=dropblock_cfg).to(device)

sum(p.numel() for p in baseline.parameters()), sum(p.numel() for p in model_db.parameters())


(2776906, 2776906)

## 6) Mini eğitim döngüsü (dummy data ile)

Gerçek veri seti indirmeden “entegrasyon doğru mu?” test etmek için random data ile 5–10 adım koşalım.

- `train()` modunda DropBlock aktif ✅
- `eval()` modunda pasif ✅


In [9]:
def train_steps(model, steps=10, bs=32, num_classes=10, lr=1e-3):
    model.train()
    opt = torch.optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    for i in range(steps):
        x = torch.randn(bs, 3, 32, 32, device=device)
        y = torch.randint(0, num_classes, (bs,), device=device)

        logits = model(x)
        loss = loss_fn(logits, y)

        opt.zero_grad(set_to_none=True)
        loss.backward()
        opt.step()

        if (i+1) % 5 == 0:
            print(f"step {i+1:02d} | loss={loss.item():.4f}")

def eval_steps(model, steps=3, bs=32, num_classes=10):
    model.eval()
    with torch.no_grad():
        for i in range(steps):
            x = torch.randn(bs, 3, 32, 32, device=device)
            logits = model(x)
    print("eval ok")

print("Baseline train:")
train_steps(baseline, steps=10)

print("\nDropBlock train:")
train_steps(model_db, steps=10)

print("\nDropBlock eval:")
eval_steps(model_db)


Baseline train:
step 05 | loss=2.2436
step 10 | loss=2.3791

DropBlock train:
step 05 | loss=2.4834
step 10 | loss=2.2723

DropBlock eval:
eval ok


## 7) İleri seviye pratik notlar (gerçek projede işine yarar)

### 7.1 Nerelere koymalısın?
- En çok fayda: **orta/derin feature map**’lerde.
- Çok erken katmanda (H,W büyükken) agresif DropBlock bazen “çok bilgi siliyor”.

**Yaygın yerleşim:**
- Residual block içinde: `conv1` sonrası veya `conv2` öncesi
- Detection modellerde: backbone’un orta/derin stage’leri

### 7.2 Hiperparametre önerileri (başlangıç)
- `block_size`: 3 / 5 / 7  
- `max_drop_prob`: 0.05 – 0.25  
- Schedule: ilk %20–30 eğitimde düşük, sonlara doğru artış

### 7.3 Küçük feature map uyarısı
H=W=7 gibi küçük haritalarda block_size büyükse agresifleşir.  
Bu notebook implementasyonu `block_size`’ı `min(block_size, H, W)` ile sınırlar.

### 7.4 BatchNorm vs GroupNorm
DropBlock, regularization olduğu için istatistikleri oynatır.  
Batch size küçükse **GroupNorm** daha stabil olabilir (biz örnekte GN kullandık).

### 7.5 Performans
DropBlock bazen:
- Top-1’i artırır ✅
- Eğitimi yavaşlatır (regularization) ✅
- Çok agresif ayarlanırsa underfit yaptırır ❌  
Bu yüzden schedule + doğru stage seçimi kritik.


# SON MODEL

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DropBlock2D(nn.Module):
    def __init__(self, drop_prob: float = 0.1, block_size: int = 7):
        super().__init__()
        # Kullanıcının verdiği "ne kadar düşüreyim" oranı
        self.drop_prob = float(drop_prob)
        # Düşürülecek blok boyutu
        self.block_size = int(block_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 1) Eğitim modunda değilsek (eval) DropBlock kapalı olmalı
        #    veya drop_prob 0 ise hiçbir şey yapmayız.
        if (not self.training) or self.drop_prob <= 0.0:
            return x

        # 2) DropBlock2D yalnızca 4 boyutlu CNN feature map bekler: (N, C, H, W)
        if x.dim() != 4:
            raise ValueError(f"DropBlock2D 4D tensor bekler (N,C,H,W). Geldi: {x.shape}")

        # 3) Tensor şekillerini al
        n, c, h, w = x.shape

        # 4) block_size, feature map boyutundan büyük olamaz.
        #    Büyükse, pratikte tüm haritayı düşürmeye gider.
        #    Bu yüzden güvenli şekilde clamp'liyoruz.
        bs = min(self.block_size, h, w)
        if bs < 1:
            return x

        # 5) Blok merkezlerini seçerken "valid" bir merkez alanı gerekir.
        #    Çünkü blok, sağa-sola genişleyecek.
        #    valid_h = H - bs + 1 -> blok tamamen sığacak merkezlerin sayısı
        valid_h = h - bs + 1
        valid_w = w - bs + 1

        # 6) Blok alanı (kaç piksel/aktivasyon düşürülecek)
        block_area = bs * bs

        # 7) gamma: "merkezlerin seçilme olasılığı"
        #    drop_prob doğrudan "piksel düşürme" değil, "blok düşürme" hedefidir.
        #    O yüzden gamma'yı, hedeflenen drop oranına göre ölçekliyoruz.
        #
        #    Bu form pratikte yaygın kullanılır:
        #    gamma = drop_prob * (H*W) / (block_area * valid_h * valid_w)
        gamma = self.drop_prob * (h * w) / (block_area * valid_h * valid_w)

        # 8) Merkez maskesi üret:
        #    Şekil: (N, C, valid_h, valid_w)
        #    Her (n,c) için olası merkez noktalarında Bernoulli örnekleriz.
        #    Seçilen merkez -> 1, seçilmeyen -> 0
        center_mask = (torch.rand((n, c, valid_h, valid_w), device=x.device) < gamma).float()

        # 9) Merkez maskesini HxW uzayına taşımamız gerekiyor.
        #    Çünkü şu an valid alan boyutunda.
        #    Bunu padding ile HxW’ye oturtuyoruz.
        #
        #    pad miktarı yaklaşık bs//2 kadar.
        pad_h = (bs - 1) // 2
        pad_w = (bs - 1) // 2
        center_mask = F.pad(center_mask, (pad_w, pad_w, pad_h, pad_h))

        # 10) Merkezleri BLOK'a genişlet:
        #     Eğer bir bölgede merkez varsa o bölgeyi 1 yapacağız.
        #     Bunu max_pool ile yapıyoruz:
        #     - kernel_size = bs -> bs x bs komşuluğa bak
        #     - stride = 1 -> her piksel konumunda uygula
        #     - padding = bs//2 -> çıktı boyutu HxW kalsın
        #
        #     Sonuç: block_mask (N,C,H,W) ve "blok alanı" 1 olur.
        block_mask = F.max_pool2d(center_mask, kernel_size=bs, stride=1, padding=bs // 2)

        # 11) Asıl uygulayacağımız maske:
        #     Drop edilen yerler 0 olmalı, tutulacak yerler 1 olmalı.
        #     block_mask'te drop edilecek blok 1 olduğu için tersliyoruz:
        mask = 1.0 - block_mask

        # 12) Ölçekleme (çok kritik):
        #     Drop sonrası aktivasyonların beklenen büyüklüğü düşer.
        #     Dropout mantığında olduğu gibi,
        #     "beklenen değer sabit kalsın" diye (1/keep_prob) ile ölçekleriz.
        keep_prob = mask.mean().clamp(min=1e-6)

        # 13) Maskeyi uygula + ölçekle
        return x * mask / keep_prob
    
class DropBlock2D_Scheduled(nn.Module):
    """
    DropBlock'u eğitim boyunca yavaş yavaş artırmak için schedule'lı sürüm.

    Neden?
    - Eğitimin başında aşırı regularization öğrenmeyi baltalayabilir.
    - Sonlara doğru artırmak daha güvenli: model önce temel temsilleri öğrenir,
      sonra daha zorlayıcı regularization ile genelleme güçlenir.

    Args:
        max_drop_prob (float): Eğitim sonunda ulaşılacak drop_prob.
        block_size (int): Blok boyutu.
        total_steps (int): Kaç adımda max_drop_prob'a ulaşacağız.
    """
    def __init__(self, max_drop_prob=0.1, block_size=7, total_steps=10_000):
        super().__init__()
        self.max_drop_prob = float(max_drop_prob)
        self.block_size = int(block_size)
        self.total_steps = int(total_steps)

        # Step sayacını buffer olarak tutuyoruz:
        # - model.state_dict() içine girer
        # - cuda/cpu taşınırken otomatik taşınır
        self.register_buffer("step", torch.zeros((), dtype=torch.long))

    def _current_drop_prob(self) -> float:
        # 1) Mevcut step'i al
        s = int(self.step.item())

        # 2) [0,1] arası progress oranı
        t = min(max(s / max(1, self.total_steps), 0.0), 1.0)

        # 3) Lineer artış: 0 -> max_drop_prob
        return self.max_drop_prob * t

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 1) Eğitim modundaysak her forward'da step arttır
        #    (istersen bunu dışarıdan da yönetebilirsin ama burada otomatik)
        if self.training:
            self.step += 1

        # 2) O anki drop_prob'u schedule'dan hesapla
        drop_prob = self._current_drop_prob()

        # 3) Eval modunda veya drop_prob 0 iken pas geç
        if (not self.training) or drop_prob <= 0.0:
            return x

        # 4) Shape kontrolü
        if x.dim() != 4:
            raise ValueError(f"DropBlock2D_Scheduled 4D tensor bekler (N,C,H,W). Geldi: {x.shape}")

        # 5) Şekilleri al
        n, c, h, w = x.shape

        # 6) Block size clamp
        bs = min(self.block_size, h, w)
        if bs < 1:
            return x

        # 7) Valid merkez alanı + blok alanı
        valid_h = h - bs + 1
        valid_w = w - bs + 1
        block_area = bs * bs

        # 8) gamma'yı drop_prob'a göre ayarla
        gamma = drop_prob * (h * w) / (block_area * valid_h * valid_w)

        # 9) Merkez maskesi örnekle
        center_mask = (torch.rand((n, c, valid_h, valid_w), device=x.device) < gamma).float()

        # 10) HxW alanına pad
        pad_h = (bs - 1) // 2
        pad_w = (bs - 1) // 2
        center_mask = F.pad(center_mask, (pad_w, pad_w, pad_h, pad_h))

        # 11) Merkezleri bloklara genişlet
        block_mask = F.max_pool2d(center_mask, kernel_size=bs, stride=1, padding=bs // 2)

        # 12) Drop mask
        mask = 1.0 - block_mask

        # 13) Ölçekleme
        keep_prob = mask.mean().clamp(min=1e-6)

        # 14) Uygula
        return x * mask / keep_prob

class ConvGNAct(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1, groups_gn=8, act="silu"):
        super().__init__()

        # 1) Conv: bias=False çünkü norm katmanı offset öğrenir
        self.conv = nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False)

        # 2) Group sayısını cout'a göre ayarla.
        #    GN'de groups, channel sayısını bölmeli.
        g = min(groups_gn, cout)
        while cout % g != 0 and g > 1:
            g -= 1

        # 3) GroupNorm
        self.norm = nn.GroupNorm(g, cout)

        # 4) Aktivasyon seçimi
        if act == "silu":
            self.act = nn.SiLU(inplace=True)
        elif act == "relu":
            self.act = nn.ReLU(inplace=True)
        else:
            raise ValueError("act 'silu' veya 'relu' olmalı")

    def forward(self, x):
        # Conv -> Norm -> Act
        return self.act(self.norm(self.conv(x)))

class ResidualBlock(nn.Module):
    def __init__(self, cin, cout, stride=1, dropblock: nn.Module | None = None):
        super().__init__()
        self.dropblock = dropblock
        self.conv1 = ConvGNAct(cin, cout, k=3, s=stride, p=1)

        self.conv2 = nn.Conv2d(cout, cout, 3, padding=1, bias=False)

        g = min(8, cout)
        while cout % g != 0 and g > 1:
            g -= 1
        self.norm2 = nn.GroupNorm(g, cout)
        self.skip = None
        if stride != 1 or cin != cout:
            self.skip = nn.Conv2d(cin, cout, 1, stride=stride, bias=False)

        self.act = nn.SiLU(inplace=True)

    def forward(self, x):
        identity = x if self.skip is None else self.skip(x)

        out = self.conv1(x)
        if self.dropblock is not None:
            out = self.dropblock(out)

        out = self.norm2(self.conv2(out))

        out = out + identity
        return self.act(out)
    
class SmallResNetDropBlock(nn.Module):
    def __init__(self, num_classes=10, dropblock_cfg=None):
        super().__init__()

        # 1) dropblock_cfg sözlüğü ile stage'lere ayrı DropBlock bağlayacağız
        dropblock_cfg = dropblock_cfg or {}
        db1 = dropblock_cfg.get("stage1", None)
        db2 = dropblock_cfg.get("stage2", None)
        db3 = dropblock_cfg.get("stage3", None)

        # 2) Stem: girişte kanal artır
        self.stem = ConvGNAct(3, 64, k=3, s=1, p=1)

        # 3) Stage 1: aynı çözünürlükte residual bloklar
        self.stage1 = nn.Sequential(
            ResidualBlock(64, 64, stride=1, dropblock=db1),
            ResidualBlock(64, 64, stride=1, dropblock=db1),
        )

        # 4) Stage 2: stride=2 ile downsample + kanal artır
        self.stage2 = nn.Sequential(
            ResidualBlock(64, 128, stride=2, dropblock=db2),
            ResidualBlock(128, 128, stride=1, dropblock=db2),
        )

        # 5) Stage 3: tekrar downsample + kanal artır
        self.stage3 = nn.Sequential(
            ResidualBlock(128, 256, stride=2, dropblock=db3),
            ResidualBlock(256, 256, stride=1, dropblock=db3),
        )

        self.pool = nn.AdaptiveAvgPool2d(1)

        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.pool(x).flatten(1)
        return self.fc(x)

if __name__ == "__main__":
    # 1) Cihaz seçimi
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 2) DropBlock olmayan baseline model
    baseline = SmallResNetDropBlock(num_classes=10, dropblock_cfg={}).to(device)

    # 3) DropBlock'lu model:
    dropblock_cfg = {
        "stage2": DropBlock2D_Scheduled(max_drop_prob=0.15, block_size=5, total_steps=2000),
        "stage3": DropBlock2D_Scheduled(max_drop_prob=0.20, block_size=7, total_steps=2000),
    }
    model_db = SmallResNetDropBlock(num_classes=10, dropblock_cfg=dropblock_cfg).to(device)

    # 4) Dummy input üret
    x = torch.randn(4, 3, 32, 32, device=device)

    # 5) Train mod: DropBlock aktif
    model_db.train()
    y = model_db(x)
    print("train out:", y.shape)

    # 6) Eval mod: DropBlock pasif
    model_db.eval()
    y2 = model_db(x)
    print("eval out:", y2.shape)